

[在colab中打开](https://colab.research.google.com/github/Latestuser/SDVN-kohya-colab-sd15/blob/main/Lora_TrainingSD15_chs.ipynb)
---

# 💡  使用指南：
步骤 1：准备用于训练的样本图片文件夹，确保图片尺寸正确，上传至 Google Drive，可以划分子文件夹。<br>
步骤 2：运行第 1 部分。<br>
步骤 3：调整 Path 以指向已上传的数据文件夹，然后运行第 2 部分。<br>
步骤 4：在第 3 部分调整训练参数，然后运行第 3 部分。<br>
步骤 5：运行第 4 部分并观察训练结果。<br>
 重要提示：每次训练时需根据情况进行相应更改。<br>
 参考值：可以保持默认设置。<br>

---




In [ ]:
# @title #1. 安装环境

from google.colab import drive
drive.mount('/content/drive')

#root_dir
root_dir = "/content"
repo_dir = f"{root_dir}/Kohya-Colab"
training_dir = f"{root_dir}/Train_Config"
pretrained_model = f"{root_dir}/Train_model"
config_dir = f"{training_dir}/Config"
#repo_dir
accelerate_config = f"{repo_dir}/accelerate_config/config.yaml"
tools_dir = f"{repo_dir}/tools"
finetune_dir = f"{repo_dir}/finetune"

!git clone https://github.com/StableDiffusionVN/SDVN-kohya-colab-sd15 {repo_dir}

%run {repo_dir}/TrainScript.ipynb

install_sd15train()

In [ ]:
# @title #2. 连接训练数据
#@markdown ##<br>🔻 2.1 Google Drive 中数据文件夹的路径
#@markdown <br>💡 `RegFolder 可以留空`
TrainFolder = "/content/drive/MyDrive/1a/new_rumiplus"  # @param {type:'string'}
RegFolder = ""  # @param {type:'string'}
#@markdown <br>💡 `Reg 图片文件夹的权重`
Prior_loss_weight = 1  # @param {type:"number"}
DataClean = False #@param {type:"boolean"}
SubFolder = True
#@markdown ##<br>🔸 2.2 自动生成标题（caption）
AutoCaption = "Waifu-1.4" # @param ["none", "BLIP-Caption", "Waifu-1.4"]
Caption_Length = "Medium" # @param ['Short','Medium','Long']

# @markdown 🟡 `添加可选标题（caption）`
Custom_Caption = "okuborumi" # @param {type:'string',placeholder:"输入可选关键字或选择 Remove 模式时要删除的字符串"}
Remove_Caption = False #@param {type:"boolean"}
Append = False
# @markdown 🟡 `自动识别子文件夹名称作为标题（caption）`
AddFolderName = False #@param {type:"boolean"}
SubFolder = True

Cap_prompt = {
    'Short':['<CAPTION>',10,30,0.8],
    'Medium':['<DETAILED_CAPTION>',10,100,0.5],
    'Long':['<MORE_DETAILED_CAPTION>',10,150,0.3]
}

Cap_extension = ".txt"
extension = Cap_extension

if DataClean == True :
    %cd {root_dir}
    clean_directory(TrainFolder)

if AutoCaption == "Waifu-1.4" :
    %cd {finetune_dir}
    config = {
        "_train_data_dir": TrainFolder,
        "batch_size": 6,
        "repo_id": "SmilingWolf/wd-v1-4-convnext-tagger-v2",
        "recursive": True,
        "remove_underscore": True,
        "general_threshold": Cap_prompt[Caption_Length][3],
        "character_threshold": 0.2,
        "caption_extension": Cap_extension,
        "max_data_loader_n_workers": 2,
        "debug": True,
        "undesired_tags": ""
    }
    final_args = f"python {finetune_dir}/tag_images_by_wd14_tagger.py {join_arg(config)}"
    print(final_args)
    !{final_args}

if AutoCaption == "BLIP-Caption" :
    %cd {finetune_dir}
    config = {
        "_train_data_dir" : TrainFolder,
        "batch_size" : 6,
        "beam_search" : True,
        "min_length" : Cap_prompt[Caption_Length][1],
        "max_length" : Cap_prompt[Caption_Length][2],
        "debug" : True,
        "caption_extension" : Cap_extension,
        "max_data_loader_n_workers" : 2,
        "recursive" : True
    }
    final_args = f"python {finetune_dir}/make_captions.py {join_arg(config)}"
    print(final_args)
    !{final_args}

if AddFolderName:
  add_forder_name(TrainFolder)
if Custom_Caption != "":
  process_dir(TrainFolder, Custom_Caption, Append, Remove_Caption)


In [ ]:
#@title #3. 设置
#@markdown ##<br>🔻 3.1 常规设置
#@markdown <br>

Train_Option = "Lora" #@param ["Checkpoint", "Lora"]
Project_name = "newrumi_a" #@param {type:"string",placeholder:"输出名称"}
Output_Path = "/content/drive/MyDrive/1A_new_rumi" #@param {'type':'string'}
#@markdown <br>💡 `粘贴下载链接或驱动器路径链接以使用任意模型进行训练`
Model_Train = "Chilloutmix"  #@param ["", "StableDiffusion_v15.ckpt", "Chilloutmix", "RealisticVision51", "PhotographyProv8", "Deliberate", "DreamShaperv8", "A-ZovyaRPGv3", "HenmixRealv4", "Fantasticv65", "XXMix_9realisticv3", "MajicmixRealv6", "ReVAnimatedv122", "RunDiffusionFXv1", "BeautifulRealAsiansv6", "LEOSAM-MoonFilm-v2","Counterfeit", "OrientalMix", "FishMix", "ThreeDelicacyv2", "GODofSIMP", "TheoldfishSpread", "Theoldfish", "NightSkyYOZORA", "GhostMixv2", "ToonYou-beta6", "ComicsArthemyv3", "MeinaMix-v11", "ManmaruMix-v2"] {allow-input: true}
Resume_Path = "" # @param {type:'string',placeholder:"如果要从备份继续训练，请输入“State”文件夹的路径名称"}
Save_State = False #@param {type:"boolean"}
#@markdown <br>💡 `如果使用子文件夹（SubFolder）进行训练，可以按照以下语法为每个子文件夹命名：{repeat}_{name}，其中：{repeat}：表示该子文件夹的重复次数。{name}：表示子文件夹的名称。例如：3_modelA 表示该子文件夹名为 modelA，并且将重复 3 次。5_dataB 表示该子文件夹名为 dataB，并且将重复 5 次。`
Repeats = 20  # @param {type:"number"}
Resolution = 512  # @param {type:"slider", min:512, max:1024, step:128}
Optimizer_type = "AdamW8bit"  # @param ["AdamW", "AdamW8bit", "Lion", "SGDNesterov", "SGDNesterov8bit", "DAdaptation", "AdaFactor"]
#@markdown <br>💡 `设置每 n 轮后的备份`
Save_Every_N_Epochs = 1  # @param {type:"number"}
#@markdown ##<br>🔻 3.2-A | Checkpoint Setting
#@markdown <br>💡 `使用 T4 显卡时，批处理大小（Batch size）的最大值为：对于尺寸 512，最大是 4；对于尺寸 768，最大是 2`
Batch_size = 2  # @param {type:"number"}
Learning_Rate = 1e-6  # @param {'type':'number'}
#@markdown <br>💡 `最大训练步数`
Max_train_steps = 2500  # @param {type:"number"}
#@markdown ##<br>🔻 3.2-B | Lora Setting
#@markdown <br>
LoRA_Network_Weights = "" # @param {'type':'string',placeholder:"输入 LoRA 的路径，如果想继续训练任意的 LoRA 模型"}
#@markdown 💡 `使用 T4 显卡时，批处理大小（Batch size）的最大值为：对于尺寸 512，最大是 6；对于尺寸 768，最大是 3`
Lora_Batch_size = 3  # @param {'type':'number'}
Network_dim = 64  # @param {'type':'number'}
Network_alpha = 32 # @param {'type':'number'}
Unet_Lr = 1e-4  # @param {'type':'number'}
Text_encoder_Lr = 1e-4  # @param {'type':'number'}
#@markdown <br>💡 `最大训练轮数`
Epochs = 5  # @param {type:"number"}
#@markdown <br>💡 `设置学习率调度器`
Lr_scheduler = "constant"  # @param ["linear", "cosine", "cosine_with_restarts", "polynomial", "constant", "constant_with_warmup", "adafactor"] {allow-input: false}
Lr_power = 2 # @param {'type':'number'}
Comment = "creator by sdvn.me" # @param {'type':'string'}


output_name = Project_name if Project_name != "" else "project_name"
resume = Resume_Path
save_state = Save_State
optimizer_type = Optimizer_type
save_every_n_epochs = Save_Every_N_Epochs
training_comment = Comment
dataset_repeats = Repeats
max_train_steps = Max_train_steps if Train_Option == "Checkpoint" else None
train_batch_size = Lora_Batch_size if Train_Option == "Lora" else Batch_size
network_weights = LoRA_Network_Weights
network_dim = Network_dim
network_alpha = Network_alpha
learning_rate = Learning_Rate if Train_Option == "Checkpoint" else None
text_encoder_lr = Text_encoder_Lr
unet_lr = Unet_Lr
max_train_epochs = Epochs if Train_Option == "Lora" else None
lr_scheduler = Lr_scheduler
lr_scheduler_power = Lr_power if Lr_scheduler == "polynomial" else None
lr_warmup_steps = 100 if Lr_scheduler == "constant_with_warmup" else None
lr_scheduler_num_cycles = 2 if Lr_scheduler == "cosine_with_restarts" else None
optimizer_args = "[ \"scale_parameter=False\", \"relative_step=False\", \"warmup_init=False\" ]" if  Optimizer_type == "AdaFactor" else None
output_dir = f'{Output_Path}/{"Lora" if Train_Option == "Lora" else "Model"}'
activation_word = output_name

vae_link = "https://huggingface.co/stabilityai/sd-vae-ft-mse-original/resolve/main/vae-ft-mse-840000-ema-pruned.ckpt"
vae_name = "vae.ckpt"
aria_down(vae_link,pretrained_model,vae_name)
vae = f"{pretrained_model}/{vae_name}"

file_path = f'{repo_dir}/model_lib.json'
with open(file_path, 'r') as json_file:
    modellist = json.load(json_file)

pretrained_model_name_or_path = download_lib(Model_Train,modellist,pretrained_model)

sample_prompt = f"{config_dir}/sample_prompt.txt"
config_file = f"{config_dir}/config_file.toml"
dataset_config = f"{config_dir}/dataset_config.toml"

default_config_lora = f"{repo_dir}/config/default_config_lora.json"
default_config_checkpoint = f"{repo_dir}/config/default_config_checkpoint.json"

final_config(default_config_lora if  Train_Option == "Lora" else default_config_checkpoint,config_file)

subsets = get_subsets(TrainFolder,RegFolder)

config = {
    "general": {
        "enable_bucket": True,
        "caption_extension": Cap_extension,
        "shuffle_caption": True,
        "keep_tokens": 0,
        "bucket_reso_steps": 64,
        "bucket_no_upscale": False,
    },
    "datasets": [
        {
            "resolution": Resolution,
            "min_bucket_reso": 320,
            "max_bucket_reso": 1280,
            "caption_dropout_rate": 0,
            "caption_tag_dropout_rate": 0,
            "caption_dropout_every_n_epochs": 0,
            "flip_aug": False,
            "color_aug": False,
            "face_crop_aug_range": None,
            "subsets": subsets,
        }
    ],
}

config_str = toml.dumps(config)
write_file(dataset_config, config_str)

dir = random.choice(subsets)["image_dir"]
txt_files = [f for f in os.listdir(dir) if f.endswith('.txt')]
sample = read_file(f"{dir}/{random.choice(txt_files)}")
default_prompt = "best quality --n worst quality, low quality, normal quality, jpeg artifacts, signature, watermark, username, blurry --w 512 --h 768 --l 7 --s 28 "
sample_str = f"{sample},{default_prompt}"
write_file(sample_prompt, sample_str)

print("Done")

In [ ]:
#@title #4.0 训练
#@markdown #▶️ 跟踪过程并控制质量
#@markdown *检查样本于 ```{Output_Path}/sample```*
#@markdown <br>*可以在任何时候对满意的样本停止训练*
AutoDisconect = False #@param {type:"boolean"}
%cd {repo_dir}

if Train_Option == "Checkpoint" :
  train_file = "train_db.py"
else :
  train_file = "train_network.py"

accelerate_conf = {
    "config_file" : accelerate_config,
    "num_cpu_threads_per_process" : 1,
}

train_conf = {
    "sample_prompts" : sample_prompt,
    "dataset_config" : dataset_config,
    "config_file" : config_file
}

accelerate_args = join_arg(accelerate_conf)
train_args = join_arg(train_conf)
final_args = f"accelerate launch {accelerate_args} {train_file} {train_args}"
print(final_args)
!{final_args}

if AutoDisconect:
    from time import sleep
    sleep(180)
    from google.colab import runtime
    runtime.unassign()

# **小工具**

In [ ]:
#@title TAG修剪

import os

# 文本文件路径
path_input = '/content/your_folder' # @param {type: "string"}

# 要添加的TAG
add_string_input = '' # @param {type: "string"}

# TAG添加的位置
position_dropdown = '开头' # @param ['开头', '结尾']

# 删除TAG
remove_string_input = '' # @param {type: "string"}

# 定义修改文件的函数
def modify_txt_files(folder_path, add_string, position, remove_string):
    try:
        if not os.path.exists(folder_path):
            print(f"错误：路径 {folder_path} 不存在")
            return

        modified_count = 0
        for filename in os.listdir(folder_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)

                # 读取文件内容
                with open(file_path, 'r', encoding='utf-8') as f:
                    content = f.read()

                # 删除指定字符串
                if remove_string:
                    content = content.replace(remove_string, '')

                # 添加字符串到开头或结尾
                if add_string:
                    if position == '开头':
                        content = add_string + content
                    else:  # 结尾
                        content = content + add_string

                # 写回文件
                with open(file_path, 'w', encoding='utf-8') as f:
                    f.write(content)

                print(f"已修改: {file_path}")
                modified_count += 1

        print(f"操作完成，共修改 {modified_count} 个文件")

    except Exception as e:
        print(f"发生错误: {str(e)}")

# 直接运行函数
modify_txt_files(
    path_input,
    add_string_input,
    position_dropdown,
    remove_string_input
)

In [ ]:
#@title TXT文件删除

import os

def delete_txt_files(folder_path):
    try:
        # 检查路径是否存在
        if not os.path.exists(folder_path):
            print(f"错误：路径 {folder_path} 不存在")
            return

        # 遍历文件夹中的所有文件
        for filename in os.listdir(folder_path):
            # 检查文件是否以.txt结尾
            if filename.endswith('.txt'):
                file_path = os.path.join(folder_path, filename)
                # 删除文件
                os.remove(file_path)
                print(f"已删除: {file_path}")

        print("操作完成")

    except Exception as e:
        print(f"发生错误: {str(e)}")

# 使用示例
# 指定要删除txt文件的文件夹路径
folder_path = "" # @param{type: "string"}
#folder_path = "/content/your_folder"  # 替换为你的实际路径
delete_txt_files(folder_path)